In [1]:
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi

c:\Users\Kavya Katta\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Kavya Katta\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


c:\Users\Kavya Katta\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [13]:
videourl = "https://youtu.be/T6FXNGynLI8?si=eTODx4Vs5Rl-vxre"

# Extract video ID from the URL
video_id = videourl.split("/")[-1]

video_id

'T6FXNGynLI8?si=eTODx4Vs5Rl-vxre'

In [14]:
from IPython.display import YouTubeVideo 

video = YouTubeVideo(video_id)

video

In [4]:
import whisper
from pytube import YouTube

def transcribe(url):
    wspr = whisper.load_model('base')
    ytrl = YouTube(url)
    streams = ytrl.streams.filter(only_audio = True)
    stream = streams.first()
    stream.download(filename = 'audio1.mp4')
    transcript = wspr.transcribe('audio1.mp4')
    transcript = transcript['text']
    return transcript



In [23]:
try:
    transcripts = YouTubeTranscriptApi.get_transcript(video_id)
    total_content = ""

    for i in transcripts:
        total_content += " " + i['text']

    print(total_content)
except:
    print("No transcript available...so creating one..")
    print('please wait for few minutes..')
    total_content = transcribe(videourl)
    print(total_content)
    

No transcript available...so creating one..
please wait for few minutes..


c:\Users\Kavya Katta\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 

In [19]:
from langdetect import detect

def detect_language(text):
    try:
        language = detect(text)
        return language
    except Exception as e:
        print(f"Error detecting language: {e}")
        return None

# Example usage
#text_to_detect = "यह तोह बोहोत ही आसान है "

detected_language = detect_language(total_content)

if detected_language:
    print(f"The detected language is: {detected_language}")
else:
    print("Language detection failed.")


The detected language is: ur


In [20]:
from googletrans import Translator
if detected_language != 'en':
    print('entered')
    translator = Translator()
    out = translator.translate(total_content,dest='en')
    total_content = out.text

print(total_content)



entered
My Sati Etober today I am very happy to welcome you as Akfilo Etober. It's been a long time since I've been there and there's a country number. There is a chance that we can make SMPEC more effective. If I am in Bish, I refuse to talk to you about some Bishas. These Bishes are those who are connected to the mass believers. The strength of the country's government is half of their strength. The Abhiyan made everyone give their contribution. The children gave it an emotional poral. The Sally British raised it. The war made it a mation in India's corner. We will not stop until this becomes the identity of thinking India, so every one of you should have a thinking animal in the priority. Another thing is that due to the success of digital payments UPI, India is currently at 46th place in the digital payments of the world. You should encourage the people of the country to make digital payments, teach them how to make digital payments in an easy way from your teacher, and one thing i

In [21]:
len(total_content.split(' '))

291

In [46]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained(
    'facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained(
    'facebook/bart-large-cnn')



In [47]:
all_summeries = []

def summarize(text, maxSummarylength=500):
    # Encode the text and summarize
    inputs = tokenizer.encode("summarize: " +
                              text,
                              return_tensors="pt",
                              max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=maxSummarylength/3*2,
                                 min_length=int(maxSummarylength/5),
                                 length_penalty=10.0,
                                 num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [17]:
def split_text_into_pieces(text,
                           max_tokens=900,
                           overlapPercent=10):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    # Calculate the overlap in tokens
    overlap_tokens = int(max_tokens * overlapPercent / 100)

    # Split the tokens into chunks of size
    # max_tokens with overlap
    pieces = [tokens[i:i + max_tokens]
              for i in range(0, len(tokens),
                             max_tokens - overlap_tokens)]

    print('pieces======>>>>>>>>>>>',pieces)

    # Convert the token pieces back into text
    text_pieces = [tokenizer.decode(
        tokenizer.convert_tokens_to_ids(piece),
        skip_special_tokens=True) for piece in pieces]

    return text_pieces


In [18]:

def recursive_summarize(text, max_length=150, recursionLevel=0):
    recursionLevel=recursionLevel+1
    print("######### Recursion level: ",
          recursionLevel,"\n\n######### ")
    tokens = tokenizer.tokenize(text)
    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2
   
    print(max_length)
    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length)

    print("Number of pieces: ", len(pieces))
    # Summarize each piece
    summaries=[]
    k=0
    for k in range(0, len(pieces)):
        piece=pieces[k]
        #print("****************************************************")
        #print("Piece:",(k+1)," out of ", len(pieces), "pieces")
        #print(piece, "\n")
        summary =summarize(piece, maxSummarylength=max_length)
        #print("SUMNMARY: ", summary)
        summaries.append(summary)
        #print("****************************************************")

    concatenated_summary = ' '.join(summaries)
    all_summeries.append(concatenated_summary)
    print('concat_summary',concatenated_summary)

    tokens = tokenizer.tokenize(concatenated_summary)
    print('length==',len(tokens))
    if len(tokens) > max_length:
        # If the concatenated_summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:
      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        '''if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)'''
        return final_summary

In [48]:
# Example usage

final_summary = recursive_summarize(total_content)

print(all_summeries)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
152
pieces======>>>>>>>>>>> [['Ġ50', '%', 'Ġof', 'ĠIndia', 'Ġis', 'Ġless', 'Ġthan', 'Ġthe', 'Ġage', 'Ġof', 'Ġ27', '.', 'ĠNow', 'Ġthis', 'Ġis', 'Ġa', 'Ġterrific', 'Ġdemographic', 'Ġdividend', 'Ġthat', 'Ġno', 'Ġgovernment', 'Ġin', 'Ġits', 'Ġright', 'Ġmind', 'Ġcan', 'Ġafford', 'Ġto', 'Ġignore', '.', 'ĠOur', 'Ġyoungsters', 'Ġtoday', 'Ġhave', 'Ġbeen', 'Ġexposed', 'Ġto', 'Ġthe', 'Ġworld', '.', 'ĠThey', 'Ġknow', 'Ġwhat', 'Ġis', 'Ġhappening', 'Ġin', 'Ġwash', 'Ġin', 'Ġsay', 'Ġwater', ',', 'Ġhygiene', 'Ġand', 'Ġsanitation', 'Ġin', 'ĠBrazil', 'Ġbecause', 'Ġyou', 'Ġtoo', 'Ġwhen', 'ĠGoogle', 'Ġall', 'Ġof', 'Ġthese', 'Ġsolutions', 'Ġconnect', 'Ġus', 'Ġlike', 'Ġnever', 'Ġbefore', 'Ġever', 'Ġin', 'Ġhistory', '.', 'ĠSo', 'Ġtherefore', 'Ġour', 'Ġyoungsters', 'Ġtoday', 'Ġhave', 'Ġthe', 'Ġability', ',', 'Ġhave', 'Ġthe', 'Ġimagination', ',', 'Ġhave', 'Ġthe', 'Ġvision', 'Ġand', 'Ġhave', 'Ġthe', 'Ġaud', 'acity', 'Ġto', 'Ġcompete', 'Ġwith', 'Ġthe', 'Ġworld', '.', 'ĠA

In [49]:
import spacy

def sentence_importance(sentence):
    return sum([word.vector_norm for word in sentence])

def summarize_paragraph(paragraph, num_sentences=3):
    nlp = spacy.load("en_core_web_sm")
    
    # Process the paragraph with spaCy
    doc = nlp(paragraph)
    
    # Get sentences and their respective importance scores
    sentences = [(sent, sentence_importance(sent)) for sent in doc.sents]
    
    # Sort sentences by importance score
    sorted_sentences = sorted(sentences, key=lambda x: x[1], reverse=True)
    
    # Get the top N sentences
    top_sentences = sorted_sentences[:num_sentences]
    
    # Sort top sentences by their original order in the paragraph
    top_sentences = sorted(top_sentences, key=lambda x: x[0].start)
    
    # Extract the text of the top sentences
    summarized_text = [sent.text.strip() for sent, _ in top_sentences]
    
    return summarized_text

# Example usage
paragraph = """
The company's financial performance in the last quarter has been outstanding. 
Revenues exceeded expectations, showing a 15% growth compared to the previous year. 
The key drivers of this success were innovative product launches and effective cost management strategies.
"""

result = summarize_paragraph(all_summeries[-1], num_sentences=5)

# Print the summarized text
for i, sentence in enumerate(result, 1):
    print(f"{i}. {sentence}")


1. 50% of India is less than the age of 27.
2. If India can solve its problem with respect to water, sanitation and hygiene, you're solving one fifth of the world problem.
3. "I don't believe that India is actually headed in the right direction and I don't feel very strong here and I feel a great deal of optimism," he says.
4. "We cannot afford as a country to not include our women" Nalkesh Sarke: "We also have combined in the same trench fiber optic cable.
5. So you can get a broadband connection with the final lake of finishing"


In [50]:
import spacy

def identify_key_phrases(text, min_length=2, max_length=4):
    try:
        # Load the English NLP model from spaCy
        nlp = spacy.load("en_core_web_sm")
        
        # Process the input text
        doc = nlp(text)
        
        # Extract key phrases (noun chunks) of specific lengths from the processed document
        key_phrases = [chunk.text for chunk in doc.noun_chunks if min_length <= len(chunk) <= max_length]
        
        return key_phrases
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
text = "Natural Language Processing is a field of study that focuses on the interaction between computers and humans using natural language."
key_phrases = identify_key_phrases(all_summeries[-1], min_length=2, max_length=4)

if key_phrases is not None:
    print("Key Phrases (lengths 2, 3, and 4):")
    print(key_phrases)


Key Phrases (lengths 2, 3, and 4):
['50%', 'the age', 'its problem', 'one fifth', 'the world problem', 'a limb', 'the right direction', 'a great deal', 'a country', 'our women', 'Nalkesh Sarke', 'a broadband connection', 'the final lake']


In [51]:
from IPython.display import HTML
def highlight_key_phrases(notes, key_phrases):
    highlighted_notes = notes
    for phrase in key_phrases:
        highlighted_notes = highlighted_notes.replace(phrase, f'\033[1;31m{phrase}\033[0m')
    return highlighted_notes

highlighted_notes = highlight_key_phrases(''.join(result), key_phrases)

# Display the highlighted notes as HTML
for i, sentence in enumerate(highlighted_notes.split('.'), 1):
    print(f"{i}. {sentence}")

1. 50% of India is less than the age of 27
2. If India can solve its problem with respect to water, sanitation and hygiene, you're solving one fifth of the world problem
3. "I don't believe that India is actually headed in the right direction and I don't feel very strong here and I feel a great deal of optimism," he says
4. "We cannot afford as a country to not include our women" Nalkesh Sarke: "We also have combined in the same trench fiber optic cable
5. So you can get a broadband connection with the final lake of finishing"
